In [0]:
from tensorflow.python.client import device_lib
import numpy as np
#device_lib.list_local_devices()

Upload the data files

In [0]:
from google.colab import files

uploaded = files.upload()

#for fn in uploaded.keys():
#  print('User uploaded file "{name}" with length {length} bytes'.format(
#      name=fn, length=len(uploaded[fn])))

Saving hackathon_training_data.zip to hackathon_training_data.zip


In [0]:
!mkdir hackathon_training_data 
!unzip hackathon_training_data.zip -d hackathon_training_data
!ls hackathon_training_data/

In [0]:
#clone the datasets repository
!\rm -r force2019-data-000; git clone https://github.com/laicheil/force2019-data-000.git;ls force2019-data-000/

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import os
import json
#data_path = os.path.join('force2019-data-000', 'data-001')#'hackathon_training_data'
data_path = os.path.join('/content/drive/My Drive/force-hackathon-2019', 'data-002') 
num_of_files = 0
list_of_files = os.listdir(data_path)
for filename in list_of_files:
  num_of_files = max (num_of_files,int(filename.split('.')[0].split('_')[-1]))
num_of_files
num_of_files *= 2
#num_of_files = len(list_of_files)
first_file_path = os.path.join(data_path, list_of_files[0])
#print (first_file_path)
with open(first_file_path,'r') as read_file:
  shape_of_files = (num_of_files,) + np.asarray(json.load(read_file)).shape + (2, )
#print (shape_of_files)


data = np.zeros((shape_of_files))
labels = np.zeros(num_of_files)
labels_ce = np.zeros((num_of_files,2))
for filename in os.listdir(data_path):

  if not filename.startswith('seismic'):
    splitted_name = filename.split('.')[0].split('_')
    if splitted_name[1] == 'seismic':
      continue
    chan = int(splitted_name[0]=='topa')
    i = int(splitted_name[-1])
    is_good = int (splitted_name[1] is 'good')
    full_path = os.path.join(data_path,filename)
    labels[i] = is_good #int(filename.startswith('good'))
    labels_ce[i, is_good] = 1
    with open(full_path,'r') as read_file:
        loaded = np.asarray(json.load(read_file))
        mask = np.zeros_like(loaded)
        mask[np.argwhere(loaded > 999990)] = 1
        data[i, :, :, chan] = loaded

print('labels shape', labels.shape)
print('labels for CE shape', labels_ce.shape)
print('data shape', data.shape)

IndexError: ignored

In [0]:
data.shape
loaded.shape
i
#data[i, :, :, chan] = loaded

'0'

In [0]:
print(labels)
print(os.listdir(data_path))



Image data generators for the inputs

In [0]:
from tensorflow.keras.preprocessing import image
from sklearn.model_selection import train_test_split

datagen = image.ImageDataGenerator (
    #featurewise_center = True,
    #featurewise_std_normalization=True,
    #samplewise_center = True,
    #samplewise_std_normalization=True,
    vertical_flip=True,
    horizontal_flip=True,
    rotation_range=90)
datagen.fit (data)

train_samples, validation_samples, train_labels, validation_labels = train_test_split(data, labels, test_size=.334)

train_generator         = datagen.flow(train_samples, train_labels, batch_size=32)
validation_generator    = datagen.flow(validation_samples , validation_labels , batch_size=32)


train_samples_ce, validation_samples_ce, train_labels_ce, validation_labels_ce = train_test_split(data, labels_ce, test_size=.334)
train_ce_generator         = datagen.flow(train_samples_ce, train_labels_ce, batch_size=32)
validation_ce_generator    = datagen.flow(validation_samples_ce , validation_labels_ce , batch_size=32)

test_ce_generator = validation_ce_generator

[link text](https://)Loading the ResNet50 model from the tensorflow-keras library

In [0]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions


import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
#config.gpu_options.per_process_gpu_memory_fraction = 0.33

from tensorflow.python.keras import backend as K
#with tf.device('/device:GPU:0'):
K.set_session (tf.Session (config = config))

print('DONE LOADING MODEL')

DONE LOADING MODEL


Callbacks

In [0]:
import tensorflow.keras.callbacks as tfkc
import datetimeimport tensorflow.keras.callbacks as tfkc

now = datetime.datetime.now ()
date_str = now.strftime('%Y%m%d%H%M')
checkpoint_init_name = 'init_chkpnt_'+date_str+'.hdf5'
from tensorflow.python.keras.callbacks import CSVLogger, EarlyStopping, ModelCheckpoint
callbacks = [ 
      
        EarlyStopping (monitor='val_acc', patience=9, verbose=1),
        ModelCheckpoint(checkpoint_init_name, monitor='val_acc', save_best_only=True, save_weights_only=True, verbose=1),
        tfkc.TensorBoard(log_dir=os.path.join('/content/drive/My Drive/force-hackathon-2019', 'tbg', now.strftime('%Y%m%d%H%M%S')), histogram_freq=0, write_graph=True, write_images=True)
        ]

In [0]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
def simpleNet (input_tensor=None):
  if input_tensor is None:
    return None
  def convBlock (units, input_tensor):
    x = Conv2D(units, (3, 3), padding='same', activation = 'relu')(input_tensor)
    x = Conv2D(units, (3, 3), activation = 'relu')(x)
    x = MaxPooling2D(pool_size=(2, 2)) (x)
    x = Dropout(0.25) (x)
    return x
  
  t = convBlock(32, input_tensor)
  t = convBlock(64, t)
  return t

In [0]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input,Lambda, Dense, Flatten, Dropout
from tensorflow.image import grayscale_to_rgb
use_resnet = False
## inputs
inputs = Input (shape=data.shape[1:])#samples.shape[1:]

# the base model
if use_resnet:
  x = Lambda (lambda x: grayscale_to_rgb (x), name='grayscale_to_rgb') (inputs)
  base_model = ResNet50(weights='imagenet', input_tensor=x,include_top=False)
  num_layers = len(base_model.layers)
  for i, layer in enumerate (base_model.layers):
    layer.trainable = i < 8 or i > num_layers-8
  # from grayscale to RGB, Xception needs 3 Channel input
  x = base_model.output
else:
  x = simpleNet(inputs)
  
# FC output layers
output = Flatten()(x)
output = Dense(512, activation='relu')(output)
output = Dropout(0.5)(output)
output = Dense(256, activation='relu')(output)
output = Dropout(0.5)(output)
output = Dense(2, activation='softmax')(output)

model = Model (inputs=inputs, outputs=output)
model.compile(optimizer='nadam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
#model.summary()

In [0]:
print(base_model.output_shape)

(None, 5, 5, 2048)


Train

Train using kfold

In [0]:
from sklearn.model_selection import KFold

k_checkpoint_basename = 'CHK_' + date_str + '_K'
kf = KFold (shuffle=True, n_splits=5)
last_good_model_weights = ''
k=0
folds_map = {}
for train_index, test_index in kf.split(data, labels_ce):
  print('At fold K=',k,' with ', len(train_index), ' samples out of total ', data.shape[0])
  kf_filepath=k_checkpoint_basename + str(k) + '.hdf5'
  callbacks[-1].filepath = kf_filepath
  history = model.fit_generator (generator       = datagen.flow(data[train_index], labels_ce[train_index], batch_size=16), 
                                 validation_data = datagen.flow(data[test_index] , labels_ce[test_index] , batch_size=16),
                                 steps_per_epoch = int(data.shape[0]/5), 
                                 epochs          = 16, 
                                 callbacks       = callbacks)
  if os.path.isfile(kf_filepath):
    #model.load_weights (kf_filepath) #Load best
    last_good_model_weights = kf_filepath
  if os.path.isfile(last_good_model_weights):
    model.load_weights (last_good_model_weights)
  evaluation = model.evaluate_generator(test_ce_generator)
  #print ('Evaluation Mean Squared Error on test data for k =', k, 'is:', evaluation*100.)
  print ('Evaluation:', evaluation)
  folds_map [k] = {
      'evaluation'   : evaluation,
      'history'      : history,
      'filepath'     : kf_filepath } 
  k += 1


In [0]:

model.load_weights('CHK_201909181440_K3.hdf5')
evaluation = model.evaluate_generator(validation_ce_generator)
predict = model.predict_generator(validation_ce_generator)

print(evaluation)
print(predict)
print(validation_labels)
type(folds_map)
print(folds_map)

[0.3450590938329697, 0.7761194]
[[0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.49734327]
 [0.50265676 0.497

In [0]:
indices = np.random.random_integers(0, train_samples.shape[0], 5)
print(indices)
test = train_samples[indices]
#model.save_weights('last_iteration.hdf5')
model.load_weights('CHK_201909181412_K0.hdf5')#checkpoint_init_name)
pred = model.predict(test)
print (pred)
print (np.argmax(pred,axis=1))
print (labels[indices])

[183  22 173  18  82]
[[0.9936487  0.00635123]
 [0.9936487  0.00635123]
 [1.         0.        ]
 [1.         0.        ]
 [0.9936487  0.00635123]]
[0 0 0 0 0]
[1. 0. 0. 0. 0.]


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: This function is deprecated. Please call randint(0, 266 + 1) instead
  """Entry point for launching an IPython kernel.
